<a href="https://colab.research.google.com/github/rameshavinash94/Custom_NER/blob/main/Avinash_Spacy_2_0_Custom_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***NOTE:I have used Spacy 2.0 for training Custom NER, we can also use Spacy 3.0 which makes it even more simpler and easier.***

**IMPORTING LIBRARIES**

In [192]:
#importing librariess
import spacy
import pandas as pd
import re
from spacy.tokens import DocBin
from tqdm import tqdm
import random

**LOADING THE INITIAL DATASET**

In [193]:
#load the provided file in a pandas df.
initial_df=pd.read_csv('/content/Summer Internship - Homework Exercise - Summer Internship - Homework Exercise.csv')

***From the Initial dataset, I'm going to seperate the train,test,validation***

In [194]:
#extract the training,test,validaiton dataset from the initial dataset
training_data = initial_df[initial_df['dataset']=='train'].drop(columns=['dataset'])
testing_data = initial_df[initial_df['dataset']=='test'].drop(columns=['dataset'])
validation_data = initial_df[initial_df['dataset']=='validation'].drop(columns=['dataset'])

In [195]:
#view the training df
training_data

,transaction_descriptor,store_number
0,DOLRTREE 2257 00022574 ROSWELL,2257
1,AUTOZONE #3547,3547
2,TGI FRIDAYS 1485 0000,1485
3,BUFFALO WILD WINGS 003,3
4,J. CREW #568 0,568
...,...,...
95,DUNKIN #355514,355514
96,THE HOME DEPOT #6828,6828
97,RUE21 #1129 BLUE,1129
98,WM SUPERCENTER #34,34


**here i'm going to perfrom text preprocessing like remove Specialcharacters and ltrip 0 in numbers.**

In [196]:
#perfrom preprocessing
all_sentences=[]
for x in training_data['transaction_descriptor']:
  tmp1 = re.sub(r"[^a-zA-Z0-9]+", ' ', x) # remove sp characters
  each_sentence=[]
  for t in tmp1.split(' '):
    if t.startswith('0'):
      each_sentence.append(t.lstrip('0'))
    else:
      each_sentence.append(t)
  all_sentences.append(' '.join(each_sentence))

In [197]:
training_data['preprocssed_data']=all_sentences

In [198]:
#view the df
training_data

,transaction_descriptor,store_number,preprocssed_data
0,DOLRTREE 2257 00022574 ROSWELL,2257,DOLRTREE 2257 22574 ROSWELL
1,AUTOZONE #3547,3547,AUTOZONE 3547
2,TGI FRIDAYS 1485 0000,1485,TGI FRIDAYS 1485
3,BUFFALO WILD WINGS 003,3,BUFFALO WILD WINGS 3
4,J. CREW #568 0,568,J CREW 568
...,...,...,...
95,DUNKIN #355514,355514,DUNKIN 355514
96,THE HOME DEPOT #6828,6828,THE HOME DEPOT 6828
97,RUE21 #1129 BLUE,1129,RUE21 1129 BLUE
98,WM SUPERCENTER #34,34,WM SUPERCENTER 34


***Now, i going to create a entity index of store number in the transaction description to train my model***

In [199]:
#create entity index
entitiy_index = [(re.search(y,x).start(),re.search(y,x).end()) for x,y, in zip(training_data.preprocssed_data,training_data.store_number)]

In [200]:
#add it to new column for easier visualization
training_data['entity_index'] = entitiy_index

In [201]:
#view th df
training_data

,transaction_descriptor,store_number,preprocssed_data,entity_index
0,DOLRTREE 2257 00022574 ROSWELL,2257,DOLRTREE 2257 22574 ROSWELL,"(9, 13)"
1,AUTOZONE #3547,3547,AUTOZONE 3547,"(9, 13)"
2,TGI FRIDAYS 1485 0000,1485,TGI FRIDAYS 1485,"(12, 16)"
3,BUFFALO WILD WINGS 003,3,BUFFALO WILD WINGS 3,"(19, 20)"
4,J. CREW #568 0,568,J CREW 568,"(7, 10)"
...,...,...,...,...
95,DUNKIN #355514,355514,DUNKIN 355514,"(7, 13)"
96,THE HOME DEPOT #6828,6828,THE HOME DEPOT 6828,"(15, 19)"
97,RUE21 #1129 BLUE,1129,RUE21 1129 BLUE,"(6, 10)"
98,WM SUPERCENTER #34,34,WM SUPERCENTER 34,"(15, 17)"


***Conver Df into required format for training***

In [202]:
#create the training data in the required format for processing
train_data=[]
for index, row in training_data.iterrows():
    train_data.append((row['preprocssed_data'],{'entities':[[row['entity_index'][0],row['entity_index'][1],'store_number']]}))

In [203]:
train_data

[('DOLRTREE 2257 22574 ROSWELL', {'entities': [[9, 13, 'store_number']]}),
 ('AUTOZONE 3547', {'entities': [[9, 13, 'store_number']]}),
 ('TGI FRIDAYS 1485 ', {'entities': [[12, 16, 'store_number']]}),
 ('BUFFALO WILD WINGS 3', {'entities': [[19, 20, 'store_number']]}),
 ('J CREW 568 ', {'entities': [[7, 10, 'store_number']]}),
 ('KRISPY KREME 40 GREENVILLE SC', {'entities': [[13, 15, 'store_number']]}),
 ('FIVE GUYS MN 1847 ECOM 612 339 9733 MN',
  {'entities': [[13, 17, 'store_number']]}),
 ('CASEYS GEN STORE 2650', {'entities': [[17, 21, 'store_number']]}),
 ('HUDDLE HOUSE 535', {'entities': [[13, 16, 'store_number']]}),
 ('JAMBA JUICE 1305', {'entities': [[12, 16, 'store_number']]}),
 ('ANN TAYLOR FACTORY 2202', {'entities': [[19, 23, 'store_number']]}),
 ('Subway 26824', {'entities': [[7, 12, 'store_number']]}),
 ('MARSHALLS 688', {'entities': [[10, 13, 'store_number']]}),
 ('OREILLY AUTO 4681', {'entities': [[13, 17, 'store_number']]}),
 ('TA 227 BARSTOW REST', {'entities': [[3, 

In [204]:
#conver it to json
convert_json = json.dumps(train_data)

In [205]:
#load the trained json in a variable
train_data= json.loads(convert_json)

In [206]:
#view the json
train_data

[['DOLRTREE 2257 22574 ROSWELL', {'entities': [[9, 13, 'store_number']]}],
 ['AUTOZONE 3547', {'entities': [[9, 13, 'store_number']]}],
 ['TGI FRIDAYS 1485 ', {'entities': [[12, 16, 'store_number']]}],
 ['BUFFALO WILD WINGS 3', {'entities': [[19, 20, 'store_number']]}],
 ['J CREW 568 ', {'entities': [[7, 10, 'store_number']]}],
 ['KRISPY KREME 40 GREENVILLE SC', {'entities': [[13, 15, 'store_number']]}],
 ['FIVE GUYS MN 1847 ECOM 612 339 9733 MN',
  {'entities': [[13, 17, 'store_number']]}],
 ['CASEYS GEN STORE 2650', {'entities': [[17, 21, 'store_number']]}],
 ['HUDDLE HOUSE 535', {'entities': [[13, 16, 'store_number']]}],
 ['JAMBA JUICE 1305', {'entities': [[12, 16, 'store_number']]}],
 ['ANN TAYLOR FACTORY 2202', {'entities': [[19, 23, 'store_number']]}],
 ['Subway 26824', {'entities': [[7, 12, 'store_number']]}],
 ['MARSHALLS 688', {'entities': [[10, 13, 'store_number']]}],
 ['OREILLY AUTO 4681', {'entities': [[13, 17, 'store_number']]}],
 ['TA 227 BARSTOW REST', {'entities': [[3, 

***SPACY TRAINING FUNCTION TO CREATE A CUSTOM NER MODEL TO TRAIN***

In [210]:
def train_spacy_nlp(train_data,iterations):
  nlp=spacy.blank('en')
  ner = nlp.create_pipe('ner')
  ner.add_label('store_number')
  nlp.add_pipe(ner, name='store_number_ner')
  disable_pipe = [pipe for pipe in nlp.pipe_names if pipe != 'store_number_ner']
  with nlp.disable_pipes(*disable_pipe):
    optimizer=nlp.begin_training()
    for iter in range(iterations):
      print('starting interation {iter}'.format(iter=iter))
      random.shuffle(train_data)
      losses={}
      for text, annotations in train_data:
        nlp.update([text],[annotations],drop=0.3,sgd=optimizer,losses=losses)
      
      print(losses)
  return (nlp)

In [211]:
#call the funciton and train the custom NER for store number
trained = train_spacy_nlp(train_data,20)

starting interation 0
{'ner': 83.34424533336271}
starting interation 1
{'ner': 4.45358080565867}
starting interation 2
{'ner': 0.00034506029689265836}
starting interation 3
{'ner': 0.0001066510684204442}
starting interation 4
{'ner': 2.5123619457029123}
starting interation 5
{'ner': 0.046725995361595524}
starting interation 6
{'ner': 0.00022261840244246377}
starting interation 7
{'ner': 1.2412164979027219e-07}
starting interation 8
{'ner': 0.008935728438180282}
starting interation 9
{'ner': 0.053156094180865456}
starting interation 10
{'ner': 7.075556834547588e-07}
starting interation 11
{'ner': 7.761524735745551e-07}
starting interation 12
{'ner': 1.8998482833730233e-09}
starting interation 13
{'ner': 0.00022252037713381154}
starting interation 14
{'ner': 2.8078532017207057e-05}
starting interation 15
{'ner': 1.916093293404672e-06}
starting interation 16
{'ner': 1.2581126631094677e-06}
starting interation 17
{'ner': 1.1752785482433454e-08}
starting interation 18
{'ner': 6.616772555792

***Perform preprocessing prior passing to our Custom NER model***

In [212]:
#this function is to perfrom porecessing prior passing to our NER model
def preprocessing(sentence):
    each_sentence=[]
    tmp1 = re.sub(r"[^a-zA-Z0-9]+", ' ', sentence) # remove sp characters
    for t in tmp1.split(' '):
      if t.startswith('0'):
        each_sentence.append(t.lstrip('0'))
      else:
        each_sentence.append(t)
    return ' '.join(each_sentence)

**RUNNING MY TRAINED MODEL ON TESTING DATA**

In [215]:
for x in testing_data['transaction_descriptor']:
  preprocess = preprocessing(x)
  doc=trained(preprocess)
  if doc.ents:
    spacy.displacy.render(doc, style="ent", jupyter=True)

**RUNNING ON VALIDATION DATA**

In [216]:
for x in validation_data['transaction_descriptor']:
  preprocess = preprocessing(x)
  doc=trained(preprocess)
  if doc.ents:
    spacy.displacy.render(doc, style="ent", jupyter=True)

From the above, we can say that, our model performed well on most occasion, even on CA142 of PAPA MURPHY and KFC stores, which was alphanumeric since it was able to train from MCDONALD'S in the dataset. 